# Bayesian Optimization: Searching for the global minima
**You should hand in a report, via Canvas, maximum 6 pages (excluding references). 
Do not attach any appendices. In the report you should present your data, models and the final results.
Do not forget to visualize your data and results.
It is important that you discuss your findings and reflect on the results.
In addition you need to hand in your Python code.
The code itself will not be graded, but it should run without errors and upon inspection reproduce the main results you present in the report.
Emphasis will be put on your ability to use appropriate terminology to describe and discuss the employed statistical methodologies and results.**

**<font color=red>DEADLINE: Dec 5 (Tuesday), 23:59</font>**

## Background
In this project you will explore and search the potential energy surface (PES) for an Au atom placed on top of a Au surface.
We will use a gold surface with Miller indices (433), which is so-called vicinal surface, corresponding to stepped (111) surface.
The Au atom  placed ontop of the surface is refered to as an ad-atom.

<img src="images/surface_433.png" alt="Drawing" style="width: 850px;"/>

Here the left image shows a side view of the surface with the ad-atom colored in red.
The right image shows the top view ($z$-direction) of the surface with the primitive cell marked as the smaller rectangle drawn in black.

Finding the position of the ad-atom with the lowest energy is a difficult problem as there are multiple local minima in the PES.
Furthermore, evaluating the potential energy is commonly done with density functional theory (DFT) which is very computationally expensive and hence it is desirable to keep the number of potential energy calculations as low as possible.
To tackle these two problems we will employ Bayesian optimization using Gaussian Processes.

Additional information on a very similar task can be found in the [paper by Todorovic *et al.*](https://doi.org/10.1038/s41524-019-0175-2), a copy of which (`Todorovic-2019.pdf` is included in this directory).

## Atomstic structure
Here, instead of DFT, we will use an embedded medium theory calculator which is provided in [`ase`](https://wiki.fysik.dtu.dk/ase/index.html) or [`asap`](https://wiki.fysik.dtu.dk/asap).
You are given a atomistic surface in `structures/surface_supercell.xyz`, which can be read as
``` python
from ase.io import read
surface = read('structures/surface_supercell.xyz')
```
When searching for the optimal location of the ad-atom we only need to consider positions inside the primitive cell since the PES will be repeating itself if we move outside the primitive cell.
We therefore only consider position in the domain
\begin{equation}
0 < x < 16.65653 \quad  \land \quad 0 < y < 2.884996
\end{equation}


## Evaluating the PES
The ad-atom energy of interest, $E$, is defined as by
\begin{equation}
E = E_\text{ad} - E_\text{surface},
\end{equation}
where $E_\text{ad}$ is the potential energy of the system with the ad-atom present and $E_\text{surface}$ the energy of just the bare surface system.
When evaluating the PES, we consider the surface to be completly rigid and only allow the ad-atom to move.
The energy can thus be written as 
\begin{equation}
E = E(x,y,z),
\end{equation}
where $(x,y,z)$ corresponds to the position of the ad-atom.
Here, we will allow the $z$-coordinate of the ad-atom to relax to its local minima (given $x$ and $y$) and thus only consider the $x-y$ dependency of the energy, meaning we write the energy $E$ as
\begin{equation}
E(x, y) = \min_z E(x, y, z).
\end{equation}
The primary goal of this project is to find the global minimum of the function $E(x, y)$ in as few evaluations of $E(x, y)$ as possible.


## Bayesian Optimization
To this end, we turn to Bayesian optimization using Gaussian processes (GPs).
The idea is to iteratively train a GP to model the energy, $E(x, y)$, and use it to suggest new possible samples.

Let $\mathbf{X}$ be an array of shape $(N, 2)$ containing $(x, y)$ points as rows and $\boldsymbol{E}$ an array of length $N$ with energies corresponding to $E(x,y)$ for each point in $\mathbf{X}$.
First we construct an initial model from a few randomly drawn samples $\mathbf{X}$ and their corresponding energies $\boldsymbol{E}$.       
This allows us to train a GP for the given $\mathbf{X}$ and $\boldsymbol{E}$, which provides both an average value $\mu$ and a standard deviation $\sigma$ for the energy at $(x, y)$.

The next step is to select a new point $(x_\text{new},y_\text{new})$ for sampling.
This is done by finding the point $(x_\text{new},y_\text{new})$ in the domain that maximizes the so-called acquisition function $A(x, y)$.
In this project, we recommend using the lower confidence  bound (LCB) acquisition function, which is defined as 
\begin{equation}
A(x, y) = -\mu (x, y) + \beta \sigma (x,y).
\end{equation}
Here, $\beta$ is a hyperparameter that can be chosen approximately from the range 1 to 5.
A large value of $\beta$ will favor more exploration whereas a smaller value will lead to more exploitation.
* **Exploitation:** Search regions with low mean (low energy)
* **Exploration:** Search regions with high uncertainty/variance
You can also adaptively adjust $\beta$ during the Bayesian optimization.
Can you think of suitable protocols?

A new data point can thus be selected by
\begin{equation}
(x_\text{new}, y_\text{new}) = \arg \max_{x, y} A(x, y)
\end{equation}
Next the energy at this new point, $E(x_\text{new}, y_\text{new})$, is evaluated and added to our samples $\mathbf{X}$ and $\boldsymbol{E}$.
This allows a new GP model to be trained and we can iterate through this procedure until convergence is (hopefully) reached.


## GPy optimization
Setting up your GP model can be achieved using a snippet such as the following
```python
k1 = GPy.kern.RBF(input_dim=2)
k2 = GPy.kern.Bias(input_dim=2)
kernel = k1 + k2
model = GPy.models.GPRegression(x_data, y_data, kernel)
model.optimize()
print(model)
```
Where `model.optimize` will find the variance, length scale and bias that maximizes the log-likelihood for the GP.
The log-likelihood may be very flat or contain multiple local minima.
In case you end up with strange parameters (such as the lengthscale being $10^{-10}$ or $10^{10}$), it can help to rerun the optimization a few times which can automatically be done via
``` python
model.optimize_restarts()
```
Additionaly, it might help to put some reasonable constraints on the parameters, via
```python
k1 = GPy.kern.RBF(input_dim=2)
k1['lengthscale'].constrain_bounded(0.1, 5)
```
Instead of strict constraints, one can also set priors over the parameters via, e.g.,
``` python
k1 = GPy.kern.RBF(input_dim=2)
k1['lengthscale'].set_prior(GPy.priors.Gamma(a=2, b=1))
```

## Helper function for z-relaxation
Below is a helper function that you can use to carry out the relaxation over $z$.
It might be helpful to also define a function only taking $(x, y)$ as argument and setting $z$ to, e.g.,
```python
z = surface.positions[:, 2].max() + 3
```
to make sure the ad-atom sits on top of the surface, and then pass $(x,y,z)$ into `calculate_adatom_energy`.

In [1]:
from ase.constraints import FixAtoms, FixedLine
from ase.optimize import BFGS
from ase import Atom
from asap3 import EMT
# from ase.calculators.emt import EMT


def calculate_adatom_energy(surface, position):
    """Adds adatom to the input surface configuration at the given position
    and relaxes the ad-atoms z-cooridnate.

    Parameters
    ----------
    bare_surface
        surface configuration without adatom
    position
        position (x, y, z) at which to insert adatom in Ångström

    Returns
    -------
    tuple comprising the energy of the adatom configuration
    """
    # add adatom
    surface_with_adatom = surface.copy()
    surface_with_adatom.append(Atom('Au', position))

    # attach calculator
    calc = EMT()
    surface_with_adatom.set_calculator(calc)

    # apply constraints
    constraints = []
    c = FixAtoms(indices=list(range(len(surface_with_adatom) - 1)))
    constraints.append(c)
    c = FixedLine(-1, [0, 0, 1])
    constraints.append(c)

    # relax configuration
    surface_with_adatom.set_constraint(constraints)
    dyn = BFGS(surface_with_adatom, logfile=None)
    dyn.run(fmax=0.02, steps=200)

    energy = surface_with_adatom.get_potential_energy()
    return energy

ImportError: DLL load failed while importing _asap: The specified module could not be found.

In [ ]:
def calculate_energy(surface, position, ad_atom):
    """Adds adatom to the input surface configuration at the given position
    and relaxes the ad-atoms z-cooridnate.

    Parameters
    ----------
    bare_surface
        surface configuration without adatom
    position
        position (x, y, z) at which to insert adatom in Ångström

    Returns
    -------
    tuple comprising the energy of the adatom configuration
    """
    
    surface = surface.copy()
    if ad_atom:
        surface.append(Atom('Au', position))

    # attach calculator
    calc = EMT()
    surface.set_calculator(calc)

    # apply constraints
    constraints = []
    c = FixAtoms(indices=list(range(len(surface) - 1)))
    constraints.append(c)
    c = FixedLine(-1, [0, 0, 1])
    constraints.append(c)

    # relax configuration
    surface.set_constraint(constraints)
    dyn = BFGS(surface, logfile=None)
    dyn.run(fmax=0.02, steps=200)

    energy = surface.get_potential_energy()
    return energy

## Setup
In this project you will need [`ase`](https://wiki.fysik.dtu.dk/ase/index.html), [`asap`](https://wiki.fysik.dtu.dk/asap) and [`GPy`](https://sheffieldml.github.io/GPy/).

In [ ]:
import ase
import asap3
import GPy

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from ase.io import read
from tqdm import tqdm
from tqdm.notebook import tqdm

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Task 1: Analyzing the PES (1.5p)
The first task of the project will be to generate and visualize the full PES, $E(x, y)$, on a grid for $(x,y)$.
This will give you access to the global minima by visual inspection and will help with understanding, e.g.,  how the optimization algorithms are working or whether they get stuck in the wrong minimum.

This task may take a few minute to run as it requires a lot of evaluations of the PES.

**Tasks:**
* Compute the PES, $E(x, y)$, on a dense grid for $x$ and $y$.
* Visualize the 2D PES as a heatmap.

In [ ]:
surface = read('structures/surface_supercell.xyz')

In [ ]:
def compute_pes(surface, xs, ys):
    z = surface.positions[:, 2].max() + 3
    E_surface = calculate_energy(surface, (0, 0), False)
    energy = np.zeros((len(xs), len(ys)))
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            position = (x, y, z)
            E = calculate_energy(surface, position, True) - E_surface
            energy[i, j] = E
    return energy

In [ ]:
n_points = 100
x_min, x_max = 0.01, 16.65653
y_min, y_max = 0.001, 2.884996
xs = np.linspace(x_min, x_max, n_points)
ys = np.linspace(y_min, y_max, n_points)

true_energy = compute_pes(surface, xs, ys)

In [ ]:
fig, ax = plt.subplots(figsize=(3.5,2.3))
cs = ax.contourf(xs, ys, true_energy.T, cmap='magma', alpha=0.6)
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel(r'E [eV]')
ax.set_xlabel(r'$x$ [Å]')
ax.set_ylabel(r'$y$ [Å]')
ax.set_xticks(np.arange(0,18,2))
ax.set_yticks([0.5,1,1.5,2,2.5])

min_energy = np.min(true_energy)
min_energy_index = np.argmin(true_energy)

grid_x, grid_y = np.unravel_index(min_energy_index, true_energy.shape)

optimal_x = xs[grid_x]
optimal_y = ys[grid_y]

print(f"The global minimum is at ({optimal_x, optimal_y})")
print(f"Variance: {np.var(true_energy)}")
true_var = np.var(true_energy)
ax.scatter(optimal_x, optimal_y, color='blue', marker='*', label='True Minimum')
ax.legend(loc='best')
ax.set_title("PES with true minimum")
plt.tight_layout()
fig.savefig("true_minimum.png")

## Task 2: Local search (3p)

In this task you will try out local search (gradient descent) and see how it succeeds or fails for our function $E(x,y)$.

For the local search consider a random starting position uniformly drawn from the $\{x,y\}$ domain.

**Tasks:**
* From visual inspection of the PES in Task 1, can you obtain a rough estimate for the percentage of local searches that would end up in the global minimum.
* Run a few (50-500) local searches and record how many ended up in the global minimum.

*Hint:* `scipy.optimize.minimize` can be used

In [ ]:
z = surface.positions[:, 2].max() + 3
E_surface = calculate_energy(surface, (0, 0), False)

def compute_pes_min(start_pos, surface):
    x, y = start_pos[0], start_pos[1]
    position = (x, y, z)
    return calculate_energy(surface, position, True) - E_surface

In [ ]:
def local_minima(x_min, x_max, y_min, y_max, n_samples): 
    x_samples = np.random.uniform(x_min, x_max, n_samples)
    y_samples = np.random.uniform(y_min, y_max, n_samples)
    
    start_pos = np.column_stack((x_samples, y_samples))
    
    mins = np.zeros_like(start_pos)
    tol=1e-6
    
    for i, sp in enumerate(start_pos):
        res = minimize(compute_pes_min, sp, args=(surface), method='L-BFGS-B', tol=tol)
        mins[i] = res.x
    return mins


In [ ]:
#print(x_opt)
#print(y_opt)
#print(np.sqrt(x_opt**2+y_opt**2))
#print(mins)
n_samples = 50
mins = local_minima(x_min, x_max, y_min, y_max, n_samples)
distances = np.sqrt(np.sum((mins - np.array([optimal_x, optimal_y]))**2, axis=1))
#print("Distances:", distances)

count = 0
for i, dist in enumerate(distances):
    if dist < 0.1:
        count += 1

print('Number of samples that reach global minimum:', count)
print(100*count/n_samples, '% of the samples that reached the global minimum.')

## Task 3: Bayesian optimization (8p)
Implement your own Bayesian optimization algorithm (using GPy for the Gaussian process part) and run it to find the global minima of $E(x, y)$.

Use the [RBF kernel](https://gpy.readthedocs.io/en/deploy/GPy.kern.src.html#module-GPy.kern.src.rbf) and include a Bias kernel if it is needed.

Suitable $\beta$ values might be in the range 1 to 5.

Start the Bayesian optimization from five (5) randomly drawn samples.        


**Tasks:**
* Given the PES in Task 1 choose and motivate priors for the lengthscale and variance parameters in the RBF kernel.
* Run Bayesian optimization and analyze how the choice of $\beta$ influences convergence. Select a good value for $\beta$.
* Rerun bayesian optimization with the selected $\beta$. How many iterations are needed until the global minimum is reached?
* Visualize the PES, the uncertainty ($\sigma$) and the acquisition function together with your samples for the final model.
* In what regions is the final GP model the most uncertain? Why?


*Hints:*
* If your algorithm gets stuck sampling the same points over and over again, try increasing $\beta$ (more exploration).         
* To find the argmax of $A(x, y)$ you can either evaluate it on a very dense grid or run multiple local searches (starting from random positions) and take the maximum obtained.

In [ ]:
def LCB(mean, var, beta):
    std_dev = np.sqrt(var)
    return -mean + beta * std_dev

# From simple_machine notebook
# helper function to convert the IG mean,mode to alpha,beta
def mean_mode_2_IG_alpha_beta(mean,mode):
    alpha = (mode + mean)/(mean-mode)
    beta = (2 * mode * mean)/(mean-mode)
    return alpha, beta

# helper function to convert the G mean,variance to alpha,beta
def mean_variance_2_G_alpha_beta(mean,variance):
    alpha = mean**2/variance
    beta = variance/mean
    return alpha, beta

In [ ]:
def gaussian_loop(X, opt_x, opt_y, beta, max_iters, with_diff = True):
    
    # Bayesian opt.
    #max_iterations = 100
    #for iteration in tqdm(range(max_iterations)):
    sampled_points = []
    sampled_energy = []
    iter = 0
    diff = 1
    while diff > 0.1:
        # Predict over the grid
        mean, var = model.predict(X)

        # Compute the acquisition function on grid.
        acquisition_values = LCB(mean, var, beta)
        #print(acquisition_values.shape)

        # Select point with maximum acquisition value
        new_point_idx = np.argmax(acquisition_values)
        new_point = X[new_point_idx, :]
        if with_diff:
            diff = np.sqrt((new_point[0] - optimal_x)**2 + (new_point[1] - optimal_y)**2)
    
        sampled_points.append(new_point)
    
        # Compute E at new point
        new_energy = compute_pes_min(new_point, surface)

        sampled_energy.append(new_energy)
    
        # Update X,E and retrain model
        model.set_XY(np.vstack([model.X, new_point]), np.vstack([model.Y, new_energy]))
        model.optimize()

        iter += 1
        if iter == max_iters:
            break

    return iter, np.array(sampled_points), np.array(sampled_energy)

In [ ]:
x_min, x_max = 0.01, 16.65653
y_min, y_max = 0.001, 2.884996
x_grid = np.linspace(x_min, x_max, n_points)
y_grid = np.linspace(y_min, y_max, n_points)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
X_grid_flat = X_grid.ravel()
Y_grid_flat = Y_grid.ravel()
X = np.column_stack((X_grid_flat, Y_grid_flat))

def gaussian_loop_old(model, beta, iterations):
    sampled_points = []
    sampled_energy = []
    for iteration in range(iterations):
        # Predict over the grid
        mean, var = model.predict(X)
    
        # Compute the acquisition function on grid.
        acquisition_values = LCB(mean, var, beta)
        #print(acquisition_values.shape)
    
        # Select point with maximum acquisition value
        new_point_idx = np.argmax(acquisition_values)
        new_point = X[new_point_idx, :]
    
        sampled_points.append(new_point)
    
        # Compute E at new point
        new_energy = compute_pes_min(new_point, surface)
    
        sampled_energy.append(new_energy)
    
        # Update X,E and retrain model
        model.set_XY(np.vstack([model.X, new_point]), np.vstack([model.Y, new_energy]))
        model.optimize()
    return np.array(sampled_points), np.array(sampled_energy)

x_min, x_max, y_min, y_max, X = init_variables(n_points)

# Initialize data.
initial_points = 5
init_X = np.column_stack((np.random.uniform(x_min, x_max, initial_points), np.random.uniform(y_min, y_max, initial_points)))
init_E = np.zeros((initial_points, 1))
for i in range(initial_points):
    init_E[i] = compute_pes_min(init_X[i,:], surface)

# Set up GP model

betas = range(1)
models = []
iters = []

l_a, l_b = mean_variance_2_G_alpha_beta(1.2,0.3)
v_a,v_b = mean_mode_2_IG_alpha_beta(true_var**2, (0.9*true_var)**2)

for beta in betas:
    # Set up GP model
    k1 = GPy.kern.RBF(input_dim=2)
    k1['lengthscale'].set_prior(GPy.priors.Gamma(a=l_a, b=l_b))
    k1['variance'].constrain_bounded(0.2*true_var, 1.8*true_var)
    #For some reason, the prior doesn't work?
    #k1['variance'].set_prior(GPy.priors.InverseGamma(a=v_a, b=1/v_b))
    #k1['lengthscale'].constrain_bounded(0.1, 5)
    k2 = GPy.kern.Bias(input_dim=2)
    kernel = k1 + k2
    model = GPy.models.GPRegression(init_X, init_E, kernel)
    model.optimize()

    iter, spec_points, spec_energies = gaussian_loop(X, optimal_x, optimal_y, beta, 50)
    print(iter)
    models.append(model)
    iters.append(iter)

In [ ]:
model_no = 1
model = models[model_no]
beta = model_no

mean, variance = model.predict(X)
std_dev = np.sqrt(variance)

acquisition_function = LCB(mean, variance, beta)

shape = int(np.sqrt(len(X)))
mean = mean.reshape((shape, shape))
std_dev = std_dev.reshape((shape, shape))
acquisition_function = acquisition_function.reshape((shape, shape))

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(7, 2.5))

cp = axs[0].contourf(X_grid, Y_grid, mean, cmap='magma', alpha=0.8)
fig.colorbar(cp, ax=axs[0], label='Energy [eV]')
axs[0].set_title('Mean (Energy)')
axs[0].scatter(model.X[:-1, 0], model.X[:-1, 1], c='red', s=2)
axs[0].scatter(model.X[-1, 0], model.X[-1, 1], c='blue', marker='*', s=100)
axs[0].set_xlabel(r'$x$ [Å]')
axs[0].set_ylabel(r'$y$ [Å]')

# Uncertainty
cp = axs[1].contourf(X_grid, Y_grid, std_dev, cmap='plasma', alpha=0.8)
fig.colorbar(cp, ax=axs[1], label='Energy [eV]')
axs[1].set_title('Uncertainty (σ)')
axs[1].scatter(model.X[:-1, 0], model.X[:-1, 1], c='red', s=2)
axs[1].scatter(model.X[-1, 0], model.X[-1, 1], c='blue', marker='*', s=100)
axs[1].set_xlabel(r'$x$ [Å]')
axs[1].set_ylabel(r'$y$ [Å]')

# Acquisition Function
cp = axs[2].contourf(X_grid, Y_grid, acquisition_function, cmap='magma', alpha=0.8)
fig.colorbar(cp, ax=axs[2], label='Energy [eV]')
axs[2].set_title('Acquisition Function (A)')
axs[2].scatter(model.X[:-1, 0], model.X[:-1, 1], c='red', s=2)
axs[2].scatter(model.X[-1, 0], model.X[-1, 1], c='blue', marker='*', s=100)
axs[2].set_xlabel(r'$x$ [Å]')
axs[2].set_ylabel(r'$y$ [Å]')

plt.tight_layout()
plt.savefig("bayes_opt_all.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3))
cp = ax.contourf(xs, ys, true_energy.T, cmap='magma', alpha=0.6)
cbar = fig.colorbar(cp)
cbar.ax.set_ylabel('Energy [eV]')

# Plot sampled points
#sampled_points_1 = np.array(sampled_points_1)  # Convert list of points to numpy array if not already
ax.scatter(model.X[:, 0], model.X[:, 1], s = 2,c='red', label='Sampled Points')

# Plot global minimum
ax.scatter(optimal_x, optimal_y, c='blue', marker='*', s=100, label='Global Minimum')
ax.set_xticks(np.arange(0,18,2))
ax.set_yticks([0.5,1,1.5,2,2.5])
ax.set_xlabel('x [Å]')
ax.set_ylabel('y [Å]')
ax.legend(loc = 'upper center', bbox_to_anchor = (0.5,-0.3))
ax.set_title('Optimization Path and Global Minimum')
plt.tight_layout()
fig.savefig("bayesian_opt_pes.png")
plt.show()


## Task 4: Transition paths barriers (5p)
In Bayesian optimization the low energy regions of the parameter space $(x, y)$ are of interest and thus get sampled more by the acquisition function.    
Next we look at using GPs as a general purpose potential/model for the full PES.
In principle once one has a fast and accurate model over the entire PES we could compute various properties of the ad-atom, such as diffusion barriers and thermodynamic properties.

Here you will look at the energy landscape along a linear path between the global minima and the local energy minimum around $(11, 2.1)$.
To construct the linear path from the initial state to the final state we can write
\begin{equation}
(x, y) = (x_{start}, y_{start}) + \lambda ( x_{end} -x_{start}, y_{end}-y_{start})
\end{equation}
where $\lambda$ should be varied between $0 \to 1$ to span the path.

In this task you will construct a general purpose GP to model the PES and compute the energy along the transition path described above.
This may require 100 to 200 or more training samples.
Use $(x_\text{start}, y_\text{start})$ and $(x_\text{end}, y_\text{end})$ plus a few random samples as a starting point and then iterativley construct a GP model using a suitable acquisition function.

**Tasks:**
* Train a general purpose GP model.
* Compute the error (e.g., [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation)) of the model evaluated over the entire PES using data from Task 1 as a function of training samples. How many samples are needed to get a "good" model in your opinion?
* Use the GP model to compute the energy and its uncertainty along the linear path between the initial and final state. Compare with the GP model obtained in Task 3, and compare with the correct energy along the path computed with EMT. Discuss your findings.


*Hint:* To train a general purpose model one can use the LCB acquisition function and let $\beta \to \infty$.

In [ ]:
# Find local minimum around x = 11, y = 2.1

loc_mins = local_minima(10,12,1.8,2.5,10)
[x_end, y_end] = np.mean(loc_mins, axis = 0)

print(f"End point is: ({x_end},{y_end}")


In [ ]:
def calc_rmse(model, xs, ys):
    #X_val = np.hstack((xs.reshape(-1,1), ys.reshape(-1,1)))
    #print(X_val.shape)
    E_predict = np.zeros((100,100))
    #This is a bit inefficient but the form is the same as for true_energy
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            E_predict[i,j], E_var = model.predict(np.array([[x,y]]))
    #print(E_predict.flatten())
    #print(true_energy.flatten())
    rmse = np.sqrt(np.sum((E_predict-true_energy)**2)/len(true_energy))
    return rmse

x_min, x_max, y_min, y_max, X = init_variables()
#print(X)
# Initialize data.
initial_points = 5
init_X = np.column_stack((np.random.uniform(x_min, x_max, initial_points), np.random.uniform(y_min, y_max, initial_points)))
init_E = np.zeros((initial_points+2, 1))
init_X = np.vstack((init_X, [x_glob, y_glob]))
init_X = np.vstack((init_X, [x_end, y_end]))
for i in range(initial_points+2):
    init_E[i] = compute_pes_min(init_X[i,:], surface)


# Set up GP model
k1 = GPy.kern.RBF(input_dim=2)
k1['lengthscale'].constrain_bounded(0.1, 5)
k2 = GPy.kern.Bias(input_dim=2)
kernel = k1 + k2
iteration_array = np.arange(10,400,10)
rmse_array = []
sampled_points = []
sampled_energies = []
gen_model = GPy.models.GPRegression(init_X, init_E, kernel)
gen_model.optimize()
for iterations in iteration_array:
    tmp_sampled_points, tmp_sampled_energies = gaussian_loop(gen_model, beta = 10000, iterations = 10)
    sampled_points.append(tmp_sampled_points)
    sampled_energies.append(tmp_sampled_energies)
    rmse = calc_rmse(gen_model,xs,ys)
    rmse_array.append(rmse)
rmse_array = np.array(rmse_array)


In [ ]:
#Plot sampled points for max iterations
sampled_points = np.array(sampled_points).reshape(-1,2)
sampled_energies = np.array(sampled_energies).reshape(-1,2)
print(sampled_points.shape)
fig, ax = plt.subplots(figsize=(3.5, 3))
cp = ax.contourf(xs, ys, true_energy.T, cmap='magma', alpha=0.6)
cbar = fig.colorbar(cp)
cbar.ax.set_ylabel('Energy [Å]')

# Plot sampled points
sampled_points = np.array(sampled_points)  # Convert list of points to numpy array if not already
ax.scatter(sampled_points[:, 0], sampled_points[:, 1], s = 1,c='red', label='Sampled Points')

# Plot global minimum
ax.scatter(x_glob, y_glob, c='blue', marker='*', s=100, label='Global Minimum')

ax.set_xlabel('x [Å]')
ax.set_ylabel('y [Å]')
ax.legend(loc='upper right')
ax.set_title('General purpose sampled points')
ax.set_xticks(np.arange(0,18,2))
ax.set_yticks([0.5,1,1.5,2,2.5])
ax.legend(loc = 'upper center', bbox_to_anchor = (0.5,-0.3))
plt.tight_layout()
fig.savefig("general_samples.png")
plt.show()


In [ ]:
# Plot RMSE as a function of number of samples
print(f"Last RMSE: {rmse_array[-1]}")

fig, ax = plt.subplots(figsize=(3.5,2.3))
ax.plot(iteration_array, rmse_array)
ax.set_title("RMSE as a function of training samples")
ax.set_ylabel("RMSE (eV)")
ax.set_xlabel("#samples")
plt.tight_layout()
fig.savefig("rmse.png")


In [ ]:
# Compare along the path with EMS and previous model
lamb = 0.2
x_lin = np.linspace(x_glob, x_end, 100).reshape(-1,1)
y_lin = np.linspace(y_glob, y_end, 100).reshape(-1,1)
points_lin = np.hstack((x_lin,y_lin))
energies_lin = []
for i in range(len(points_lin)):
    energies_lin.append(compute_pes_min(points_lin[i], surface))
energies_lin = np.array(energies_lin)

dists = np.sqrt((x_lin-x_glob)**2 + (y_lin-y_glob)**2)



In [ ]:
dists = dists.flatten()
gen_E_lin, gen_E_lin_var = gen_model.predict(points_lin)
mod_E_lin, mod_E_lin_var = model.predict(points_lin)
rmse_gen_true = np.sqrt(np.sum((gen_E_lin.flatten()-energies_lin)**2)/len(energies_lin))
rmse_mod_true = np.sqrt(np.sum((mod_E_lin.flatten()-energies_lin)**2)/len(energies_lin))

print(f"RMSE for first model: {rmse_mod_true} eV")
print(f"RMSE for general model: {rmse_gen_true} eV")

print(f"True minimum = ({optimal_x}, {optimal_y})")
print(f"Specific model minimum = ({x_glob}, {y_glob})")
print(f"Generic model minimum = ({x_lin[np.argmin(gen_E_lin)]}, {y_lin[np.argmin(gen_E_lin)]})")

fig, ax = plt.subplots(figsize=(3.5,4))
ax.plot(dists,energies_lin, '--',label="true energy", color = 'r', zorder = 5)
ax.fill_between(dists,(gen_E_lin-gen_E_lin_var).flatten(), (gen_E_lin+gen_E_lin_var).flatten(), color = 'g', alpha = 0.5, zorder = 3)
ax.plot(dists,gen_E_lin, label = "general model", color = 'g', zorder = 4)
ax.fill_between(dists,(mod_E_lin-mod_E_lin_var).flatten(), (mod_E_lin+mod_E_lin_var).flatten(), color = 'b', alpha = 0.5, zorder = 1)
ax.plot(dists,mod_E_lin, label = "first model", color = 'b', zorder = 2)
ax.legend(loc = 'upper center', bbox_to_anchor = (0.5,-0.2))
ax.set_title("True and predicted energies along line")
ax.set_ylabel("Energy [eV]")
ax.set_xlabel("Distance [Å]")
plt.tight_layout()
plt.savefig("path_energy.png")